# Generate Figure of Merit based dataset

This notebook is to generate a set of DR2-lite style `.par` and `.tim` files based on a figure of merit (FoM).
We include functions for a burst with memory (BWM) and stochastic gravitational wave background (gwb) FoM.

In [ ]:
from __future__ import division, print_function, unicode_literals

import numpy as np
import os, glob
import json
import matplotlib.pyplot as plt
import libstempo as t2
from enterprise.signals import utils

import dr2lite_utils as dr2u

%matplotlib inline

Update paths to reflect your usage.

In [ ]:
DR2dir = os.path.abspath('/Users/ptb/Projects/pulsar/data/DR2')
datadir = os.path.join(DR2dir, 'release/VersionA')

lite_dir = os.path.join(DR2dir, 'DR2lite_partim')  # cleaned/combined par/tim files
os.system('mkdir -p {}'.format(lite_dir));

# 1) clean up `.par` and `.tim` files

## make list of pulsar names for later

In [ ]:
psrlist = []
psrs = glob.glob(datadir + '/J*')
for psr in psrs:
    name = psr.split('/')[-1]
    psrlist.append(name)
psrlist.sort()

## clean `.par` files

In [ ]:
parfiles = glob.glob(datadir + '/J*/*IPTADR2.par')

for p in parfiles:
    name = p.split('/')[-2]
    outfile = os.path.join(lite_dir, '{}.par'.format(name))
    dr2u.clean_par(p, outfile)

## combine `.tim` files

In [ ]:
timfiles = glob.glob(datadir + '/J*/*IPTADR2.tim')

for t in timfiles:
    name = t.split('/')[-2]
    outfile = os.path.join(lite_dir, '{}.tim'.format(name))
    dr2u.combine_tim(t, outfile)

# 2) Figure of Merit Calculation

Calculate the FoM for each pulsar for each PTA that observes it.
Both of these FoMs are for white noise dominated pulsars, which is not true... You may want to modify them.

## Burst with Memory FoM
From van Haasteren & Levin (2010) the uncertainty on the measurement of the BWM amplitude is estimated for even cadence, white-noise dominated observations.  Madison *et al*. (2014) interpret this as the minimum amplitude for a 1-$\sigma$ detection.

$$ \sigma_{h_0} = h_{min} \sim \sigma_\star \,\Delta t^{1/2}\, T^{-3/2}, $$

where $\sigma_\star$ is the TOA measurment uncertainty for a single pulsar or the response weighted root-mean-squared of uncertainty for all pulsars in a network, $\Delta t$ is the observing cadence, and $T$ the observation baseline.

To maximize the sensitivity of our array we want to minimize $\sigma_{h_0}$.
We define our figure of merit for pulsar selection as:
$$ \mathrm{FoM}_{bwm} = \frac{1}{\sigma_{h_0}} = \sqrt{\frac{T^{3}}{\left<{\sigma_{TOA}}\right>^2 \, \Delta t}} $$

## Gravitational Wave Background FoM

From GWB signal-to-noise ratio (SNR) scaling laws... used at the December 2017 busyweek at Flatiron CCA in NYC.

$$ \mathrm{FoM}_{gwb} = \frac{T^\frac{13}{3}}{\left<{\sigma_{TOA}}\right>^2 \, \Delta t} $$

And of the intermediate regime:

$$ \mathrm{FoM}_{gwb} = T^\frac{1}{2}\left(\frac{1}{\left<{\sigma_{TOA}}\right>^2 \, \Delta t}\right)^\frac{1}{2\beta} $$

In [ ]:
def bwm_FoM(Tobs, dt, TOAerr):
    """BWM FoM = 1/sig_h0
    """
    return np.sqrt(Tobs**3/dt) / TOAerr

def gwb_FoM_weak(Tobs, dt, TOAerr):
    """GWB FoM from SNR scaling laws
    """
    return Tobs**(13/3) / TOAerr**2 / dt 

def gwb_FoM_inter(Tobs, dt, TOAerr):
    """GWB FoM from SNR scaling laws
    """
    return Tobs**(1/2) * (TOAerr**2 * dt) ** (1/(2*13/3)) 

my_FoM = bwm_FoM  # function alias for FoM of choice

In [ ]:
# analysis tag to use in filenames
tag = "bwm_10d"

# filter parameters
BW = 1.1
DM_window = 10

# best PSR selection parameters
Tmin = 5.0  # yrs
sigmax = 10.0  # us

In [ ]:
ptas = ['PPTA', 'EPTA', 'NANOGrav']
backends = {'NANOGrav': 
                ['327_ASP', '430_ASP', 'L-wide_ASP', 'S-wide_ASP',
                 '327_PUPPI', '430_PUPPI', 'L-wide_PUPPI',  'S-wide_PUPPI',
                 'Rcvr_800_GASP', 'Rcvr1_2_GASP',
                 'Rcvr_800_GUPPI', 'Rcvr1_2_GUPPI',
                ],
            'PPTA': 
                ['PDFB_10CM', 'PDFB_20CM', 'PDFB_40CM',
                 'CPSR2_20CM', 'CPSR2_50CM',
                 'WBCORR_10CM', 'WBCORR_20CM',
                ],
                
            'EPTA': 
                ['EFF.EBPP.1360', 'EFF.EBPP.1410', 'EFF.EBPP.2639',
                 'JBO.DFB.1400', 'JBO.DFB.1520',
                 'NRT.BON.1400', 'NRT.BON.1600', 'NRT.BON.2000',
                 'WSRT.P1.328', 'WSRT.P1.328.C',
                 'WSRT.P1.382', 'WSRT.P1.382.C',
                 'WSRT.P1.840', 'WSRT.P1.840.C',
                 'WSRT.P1.1380', 'WSRT.P1.1380.C',
                 'WSRT.P1.1380.1',
                 'WSRT.P1.1380.2', 'WSRT.P1.1380.2.C',
                 'WSRT.P1.2273.C',
                ]
           }  # backends to use for each PTA, values for `-group` flag

### Backend Notes:
#### NANOGrav

legacy, not used (`-group` flag)
```
kaspi23 (-be mk3_23m, mk2_23)
kaspi14 (-be mk3_14m, mk2_14, mk3_14r)
L-band_Mark3a
L-band_Mark3b
L-band_Mark4
S-band_Mark4
L-band_ABPP
S-band_ABPP
```

#### PPTA

backend list for `-f` flag
```
['1050CM_PDFB1', '1050CM_PDFB2', '1050CM_PDFB3', '1050CM_PDFB4',
 '1050CM_WBCORR',
 'H-OH_CPSR2m', 'H-OH_CPSR2n', 'H-OH_PDFB1',
 'MULTI_CPSR2m', 'MULTI_CPSR2n',
 'MULTI_PDFB1', 'MULTI_PDFB2', 'MULTI_PDFB3', 'MULTI_PDFB4',
 'MULTI_WBCORR',
]
```
small number of TOAs
```
PDFB_50CM -- J0437 & J1045, 2 TOAs total
WBCORR_20CM -- 11 Parkes PSRs, 30 TOAs for J0437, rest with <10 (still used... should it be?)
```
there are some backends with `legacy` in the name... I'm not using those.  They have lots of missing metadata.
```
PKS.fptm.20cm-legacy
PKS.s2.20cm-legacy
PKS.cpsr1.20cm_legacy
PKS.cpsr2.20cm_legacy
PKS.AFB.20cm_legacy
PKS.cpsr2.50cm_legacy
```

#### EPTA

possible legacy, only used for single freq J1939
```
NRT.DDS.1400
```
small number of TOAs
```
JBO.DFB.5000 -- J1713, 4 TOAs

```

### Construct `FoM_dict` (takes a long time, skip if already saved)

Runs `filter_psr()` for each pulsar for each PTA, computing the FoM.

In [ ]:
redo = False

dict_file = 'FoM_output/FoM_{}_dict.json'.format(tag)  # where to save FoM dictionary
data_file = 'FoM_output/FoM_{}.dat'.format(tag)  # plain text file of FoM data

In [ ]:
if redo or not os.path.isfile(dict_file):
    FoM_dict = {}
    fout = open(data_file, 'w')
    fout.write('{:11}{:9}{:9}{:9}{:9}{:9}{:12}\n'
               .format(' Pulsar', ' PTA', ' raw_Tobs', '  mf_Tobs', '  TOAerr', '  cadence', '      FoM'))

    for psrName in psrlist:
        FoM_dict[psrName] = {}
    
        parfile = os.path.join(lite_dir, '{}.par'.format(psrName))
        timfile = os.path.join(lite_dir, '{}.tim'.format(psrName))
        psr = t2.tempopulsar(parfile, timfile, maxobs=30000)
        for pta in ptas:
            if pta in psr.flagvals('pta'):
                FoM_dict[psrName][pta] = {}
                filt = {'pta':[pta], 'group':backends[pta]}
                psr = dr2u.filter_psr(psr, bw=BW, dt=DM_window, filter_dict=filt, plot=False)
                
                idx = ~psr.deletedmask()
                Tobs_raw = (psr.toas().max()-psr.toas().min()) / 365.25
                if len(psr.toas()[idx]) > 0:
                    U, _ = utils.create_quantization_matrix(psr.toas()[idx]*86400, dt=86400, nmin=1)
                    cadence = (psr.toas()[idx].max() - psr.toas()[idx].min()) / U.shape[1] # mean dt
                    Tobs = (psr.toas()[idx].max()-psr.toas()[idx].min()) / 365.25
                    sigma = 1 / np.mean(1/psr.toaerrs[idx])  # harmonic mean TOAerr
                    FoM = my_FoM(Tobs, cadence, sigma)
                else:
                    Tobs = 0
                    sigma = np.inf
                    cadence = np.inf
                    FoM = 0
                fout.write('{:11}{:9}{:9.2f}{:9.2f}{:9.2f}{:9.2f}{:9.2f}\n'
                            .format(psrName, pta, Tobs_raw, Tobs, sigma, cadence, FoM))
                FoM_dict[psrName][pta]['Tobs'] = Tobs
                FoM_dict[psrName][pta]['dt'] = cadence
                FoM_dict[psrName][pta]['sigma'] = sigma
                FoM_dict[psrName][pta]['FoM'] = FoM
        del psr
    fout.close()

### save `FoM_dict` as `json` database for later usage

In [ ]:
if redo or not os.path.isfile(dict_file):

    # convert FoM_dict to python float (json can't handle numpy.float128)
    for psr in FoM_dict.keys():
        for pta in FoM_dict[psr].keys():
            for key in FoM_dict[psr][pta].keys():
                FoM_dict[psr][pta][key] = float(FoM_dict[psr][pta][key])

    with open(dict_file, 'w') as fout:
        json.dump(FoM_dict, fout, sort_keys=True, indent=4, separators=(',', ': '))

# 3) Select pulsars to use

Use only PSRs with `Tmin` yrs of multi-frequency data.  Pick one PTA per pulsar.

In [ ]:
#dict_file = 'FoM_output/FoM_{}_dict.json'.format(tag)  # FoM dictionary, same as above
with open(dict_file, 'r') as fin:
    FoM_dict = json.load(fin)

In [ ]:
psrdict = {}

for psrname in psrlist:
    best = {'pta':None, 'Tobs':0, 'FoM':0}
    for pta in ptas:
        try:
            this = FoM_dict[psrname][pta]
            if this['Tobs'] > Tmin and this['sigma'] < sigmax and this['FoM'] > best['FoM']:
                best = {'pta':pta, 'Tobs':this['Tobs'], 'FoM':this['FoM']}
        except KeyError:
            pass
    if best['pta'] is not None:
        psrdict.update({psrname: {'pta':[best['pta']], 'group':backends[best['pta']]}})
        print('{}: {pta:8}, T = {Tobs:5.2f} yr, FoM = {FoM:6.2f}'.format(psrname, **best))

In [ ]:
E,N,P = 0,0,0
for val in psrdict.values():
    val = val['pta']
    if 'EPTA' in val:
        E += 1
    elif 'NANOGrav' in val:
        N += 1
    elif 'PPTA' in val:
        P += 1
print(' EPTA - {:2d}\n   NG - {:2d}\n PPTA - {:2d}'.format(E,N,P))

In [ ]:
# save `psrdict` to file
fname = 'psrdicts/FoM_{}_psrs.json'.format(tag)
with open(fname, 'w') as fout:
    json.dump(psrdict, fout, sort_keys=True, indent=4, separators=(',', ': '))

# 3) Create filtered par and tim files for DR2-lite analysis

In [ ]:
# directory where you want the dataset saved... change for your usage
rundir = '/Users/ptb/Projects/pulsar/11yr/bwm_results/dr2lite/partim_bwm'

dr2u.make_dataset(psrdict, indir=lite_dir, outdir=rundir,
                  bw=BW, dt=DM_window, plot=False)